In [ ]:
pip install tensorflow numpy matplotlib scikit-learn opencv-python


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load and preprocess data
def load_data(image_folder, label_folder):
    images = []
    labels = []
    for file_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, file_name)
        label_path = os.path.join(label_folder, file_name.replace('.jpg', '.png'))  # Adjust extension as needed
        
        # Load image and label
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Resize for CNN input
        label = cv2.imread(label_path, 0)  # Load ground truth as grayscale
        
        images.append(img)
        labels.append(1 if label.sum() > 0 else 0)  # Binary label (Cancer/No Cancer)

    return np.array(images), np.array(labels)

# Paths to your dataset
image_folder = "D:\\prostate cancer\\Input Images-20240403T134056Z-001\\Input Images"
label_folder = "D:\\prostate cancer\\Ground Truth-20240403T133821Z-001\\Ground Truth"

images, labels = load_data(image_folder, label_folder)
images = images / 255.0  # Normalize images

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True, zoom_range=0.2)
datagen.fit(X_train)

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    epochs=20)

# Save the model
model.save("prostate_cancer_model.h5")

# Plot training history
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()
